In [4]:
import neuralNetworkCarsSystem.AutoAssistant as aa
from getpass import getpass

Для начала надо поднять Elasticsearch.

docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.12.1

И создать индекс:

curl -X PUT "http://localhost:9200/langchain_index"

А если надо перезапустить базу, то можно удалить индекс и создать снова:

curl -X DELETE 'http://localhost:9200/langchain_index' 

Инициализируем api

In [7]:
api = aa.OpenAIApi(getpass("username without domain:"), getpass("password:"))

In [8]:
from elasticsearch import Elasticsearch
# Этот код добавляет к индексу анализатор, чтобы не учитывался регистр
es_client = Elasticsearch(["http://localhost:9200"])

index_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "keyword_lowercase": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase"]
                }
            }
        }
    }
}

es_client.indices.close(index="langchain_index")
es_client.indices.put_settings(index="langchain_index", body=index_settings)
es_client.indices.open(index="langchain_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

Создаем бд, считываем и загружаем доки. n - количество разбиений при загрузке, sleep_secons - сколько секунд ждем между загрузками. Это необходимо, так как есть лимит токенов в минуту.

lazily=Truе - это если дебажишь и уже все загрузил в бд. После перезапуска данные из базы не удаляются, поэтому можно просто засунуть данные в класс и все. Но так суммаризация будет делаться после запросов, не успел сделать так, чтобы суммаризация тоже в бд сохранялась.

In [24]:
import pandas as pd

pd.read_excel("cars_pred.xlsx").shape

(5812, 8)

In [22]:
db = aa.OpenAiElasticsearchDB(api)
docs, ids = aa.get_docs("../cars_pred.xlsx")
db.add_documents(documents=docs, ids=ids, n=10, sleep_seconds=0, lazily=True) 

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc7 in position 10: invalid continuation byte

In [14]:
db.similarity_search("Мне нужна переднеприводная машина до 2 млн до 2020 года")

[Document(metadata={'id': '671b41afb7fbcf9ec5184183', 'start_year': 2011, 'end_year': 2013, 'price': 1915000.0, 'brand': 'Audi', 'model': 'A5', 'country': 'Германия', 'drive': 'Передний', 'engine_type': 'бензин', 'fuel_consumption': 6.2, 'seats': 4, 'body_type': 'Открытый кузов', 'doors': 2, 'transmission': 'вариатор', 'horsepower': 211, 'clearance': 120, 'rating': 9.3, 'desc_summarization': 'A5 Cabrio — это стильный четырехместный кабриолет, предлагаемый российскими дилерами с разнообразными бензиновыми двигателями мощностью от 180 до 265 л.с. Автомобиль оснащен различными типами трансмиссий, включая 6-ступенчатую механическую коробку передач, вариатор Multitronic и роботизированную коробку передач S-tronic. Привод может быть как передним, так и полным, что делает его универсальным для различных дорожных условий.', 'desc_plus': 'Одним из главных преимуществ A5 Cabrio является его элегантный дизайн и возможность наслаждаться открытым небом благодаря классической тканевой крыше, которая

In [5]:
bot = aa.AutoAssistant(db)

while True:
    q = input("Enter your query: ")
    if q == 'EXIT':
        break
    elif q == 'RESET':
        bot.reset()
    else:
        results = bot.post_query(q, 3)
        if len(results) == 0:
            print('No results found')
        for res in results:
            print(res.metadata)
            print(res.metadata['brand'], res.metadata['model'])
            # print(bot.get_summarization(res.metadata['id']))

In [11]:
filter = aa.OpenAiElasticsearchFilter(api)

while True:
    #q = input("Enter your query: ")
    q = "Мне нужна переднеприводная машина"
    if q == 'EXIT':
        break
    elif q == 'RESET':
        filter.reset()
    else:
        res = filter.post_query(q)
        display(res)
    break

[{'bool': {'must': [{'range': {'metadata.start_year': {'gte': 0}}},
    {'range': {'metadata.end_year': {'lte': 1000000000}}},
    {'range': {'metadata.price': {'gte': 0, 'lte': 1000000000}}},
    {'bool': {'should': []}},
    {'bool': {'should': []}},
    {'bool': {'should': [{'match': {'metadata.drive': {'query': 'передний',
          'analyzer': 'keyword_lowercase'}}}]}},
    {'bool': {'should': []}},
    {'range': {'metadata.fuel_consumption': {'gte': 0, 'lte': 1000000000}}},
    {'range': {'metadata.seats': {'gte': 0, 'lte': 1000000000}}},
    {'bool': {'should': []}},
    {'range': {'metadata.doors': {'gte': 0, 'lte': 1000000000}}},
    {'bool': {'should': []}},
    {'range': {'metadata.horsepower': {'gte': 0, 'lte': 1000000000}}},
    {'range': {'metadata.clearance': {'gte': 0, 'lte': 1000000000}}}]}}]